# Analysis of German Used Car Sales from eBay Classifieds
The aim of this project is to clean the data and analyze the included used car listings.

The data dictionary provided with data is as follows:

1. dateCrawled - When this ad was first crawled. All field-values are taken from this date.
2. name - Name of the car.
3. seller - Whether the seller is private or a dealer.
4. offerType - The type of listing
5. price - The price on the ad to sell the car.
6. abtest - Whether the listing is included in an A/B test.
7. vehicleType - The vehicle Type.
8. yearOfRegistration - The year in which the car was first registered.
9. gearbox - The transmission type.
10. powerPS - The power of the car in PS (horse power).
11. model - The car model name.
12. kilometer - How many kilometers the car has driven.
13. monthOfRegistration - The month in which the car was first registered.
14. fuelType - What type of fuel the car uses.
15. brand - The brand of the car.
16. notRepairedDamage - If the car has a damage which is not yet repaired.
17. dateCreated - The date on which the eBay listing was created.
18. nrOfPictures - The number of pictures in the ad.
19. postalCode - The postal code for the location of the vehicle.
20. lastSeenOnline - When the crawler saw this ad last online.

### Load Workspace

In [1]:
import re
import datetime as dt

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Load the Data

In [2]:
df = pd.read_csv('autos.csv', encoding='latin')
df.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

### Data Cleaning

**Rename Columns**

In [4]:
new_columns = {
    'dateCrawled':'date_crawled',  
    'offerType':'offer_type', 
    'price':'price_usd',
    'vehicleType':'vehicle_type', 
    'yearOfRegistration':'registration_year', 
    'powerPS':'power_ps', 
    'odometer':'odometer_km',
    'monthOfRegistration':'registration_month', 
    'fuelType':'fuel_type', 
    'notRepairedDamage':'unrepaired_damage', 
    'dateCreated':'date_created', 
    'nrOfPictures':'num_of_pictures', 
    'postalCode':'postal_code', 
    'lastSeen':'last_seen'
}

df = df.rename(columns=new_columns)
df.head()

,date_crawled,name,seller,offer_type,price_usd,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,date_created,num_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [5]:
df.dtypes

date_crawled          object
name                  object
seller                object
offer_type            object
price_usd             object
abtest                object
vehicle_type          object
registration_year      int64
gearbox               object
power_ps               int64
model                 object
odometer_km           object
registration_month     int64
fuel_type             object
brand                 object
unrepaired_damage     object
date_created          object
num_of_pictures        int64
postal_code            int64
last_seen             object
dtype: object

**Clean Values**

In [6]:
df.columns

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price_usd', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer_km', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'date_created', 'num_of_pictures', 'postal_code',
       'last_seen'],
      dtype='object')

In [7]:
df = df.assign(
    date_crawled=lambda x: pd.to_datetime(x.date_crawled),
    date_created=lambda x: pd.to_datetime(x.date_created),
    last_seen=lambda x: pd.to_datetime(x.last_seen),
    postal_code=lambda x: x.postal_code.astype(str),
    seller=lambda x: np.where(
        x.seller=='privat', 'private', 'commercial'
    ),
    name=lambda x: x.name.str.replace('_', ' '),
    offer_type=lambda x: np.where(
        x.offer_type=='Angebot', 'offer', 'request'
    ),
    price_usd=lambda x: x.price_usd.str.replace('$', '', regex=False).str.replace(',','', regex=False).astype(float),
    odometer_km=lambda x: x.odometer_km.str.replace('km', '', regex=False).str.replace(',','', regex=False).astype(float),
    gearbox=lambda x: np.where(
        x.gearbox.notna(),
        np.where(x.gearbox=='manuell', 'manual', 'automatic'),
        x.gearbox
    ),
    vehicle_type=lambda x: x.vehicle_type.str.replace('kleinwagen', 'small car').str.replace('kombi', 'station wagon').str.replace('cabrio', 'convertible').str.replace('andere', 'others'),
    fuel_type=lambda x: x.fuel_type.str.replace('benzin', 'petrol').str.replace('elektro', 'electric').str.replace('andere', 'others'),
    unrepaired_damage=lambda x: x.unrepaired_damage.str.replace('nein', 'no').str.replace('ja', 'yes'),
    model=lambda x: x.model.str.replace('andere', 'others')
)

In [8]:
df.dtypes

date_crawled          datetime64[ns]
name                          object
seller                        object
offer_type                    object
price_usd                    float64
abtest                        object
vehicle_type                  object
registration_year              int64
gearbox                       object
power_ps                       int64
model                         object
odometer_km                  float64
registration_month             int64
fuel_type                     object
brand                         object
unrepaired_damage             object
date_created          datetime64[ns]
num_of_pictures                int64
postal_code                   object
last_seen             datetime64[ns]
dtype: object

**Handling Missing Values**

Let's drop the `number of pictures` feature since there are no unique values:

In [9]:
df.num_of_pictures.value_counts(dropna=False)

0    50000
Name: num_of_pictures, dtype: int64

In [10]:
df = df.drop(columns='num_of_pictures')

Let's drop records where `vehicle type` and `price_usd` are missing. These are most likely improper car listings:

In [11]:
drop_index = df.loc[(df.vehicle_type.isna()) & (df.price_usd<=100)].index
df = df.drop(index=drop_index)

In [12]:
df.isnull().sum()

date_crawled             0
name                     0
seller                   0
offer_type               0
price_usd                0
abtest                   0
vehicle_type          4384
registration_year        0
gearbox               2272
power_ps                 0
model                 2524
odometer_km              0
registration_month       0
fuel_type             4019
brand                    0
unrepaired_damage     9290
date_created             0
postal_code              0
last_seen                0
dtype: int64

Upon review, most of the missing `vehicle_type` are small cars. We'll replace the missing values with `small car`:

In [13]:
df.loc[df.vehicle_type.isna(), ['name', 'vehicle_type', 'price_usd', 'gearbox']]

,name,vehicle_type,price_usd,gearbox
8,Seat Arosa,NaN,250.0,manual
10,VW Golf Tuning in siber/grau,NaN,999.0,manual
11,Mercedes A140 Motorschaden,NaN,350.0,NaN
46,BMW mit Lpg,NaN,9000.0,automatic
59,Mercedes A Klasse W 168 A 140 gruen,NaN,700.0,manual
...,...,...,...,...
49926,Vw golf cabrio,NaN,850.0,NaN
49935,Mercedes A klasse angemeldet mit Tuef und Auto...,NaN,800.0,automatic
49938,Mercedes Benz A 160 Avantgarde,NaN,2300.0,automatic
49985,Verkaufe meinen vw vento!,NaN,1000.0,automatic


In [14]:
df.loc[df.vehicle_type.isna(), 'vehicle_type'] = 'small car'

Fill `gearbox` missing values with the mode:

In [15]:
df.gearbox.mode()

0    manual
Name: gearbox, dtype: object

In [16]:
df.loc[df.gearbox.isna(), 'gearbox'] = 'manual'

Update missing values in `fuel_type` with mode:

In [17]:
df.fuel_type.value_counts()

petrol      29929
diesel      14507
lpg           688
cng            71
hybrid         37
others         20
electric       18
Name: fuel_type, dtype: int64

In [18]:
df.loc[df.fuel_type.isna(), 'fuel_type'] = 'petrol'

Update missing values in `unrepaired_damage` with mode:

In [19]:
df.unrepaired_damage.value_counts(dropna=False)

no     35136
NaN     9290
yes     4863
Name: unrepaired_damage, dtype: int64

In [20]:
df.loc[df.unrepaired_damage.isna(), 'unrepaired_damage'] = 'no'

Update missing values in `model` with `others`:

In [21]:
df.model.value_counts(dropna=False)

golf          3954
others        3490
3er           2741
NaN           2524
polo          1721
              ... 
kalina           2
rangerover       1
i3               1
200              1
b_max            1
Name: model, Length: 246, dtype: int64

In [22]:
df.loc[df.model.isna(), 'model'] = 'others'

Check for missing values:

In [23]:
df.isnull().sum()

date_crawled          0
name                  0
seller                0
offer_type            0
price_usd             0
abtest                0
vehicle_type          0
registration_year     0
gearbox               0
power_ps              0
model                 0
odometer_km           0
registration_month    0
fuel_type             0
brand                 0
unrepaired_damage     0
date_created          0
postal_code           0
last_seen             0
dtype: int64

Separate date and time from `date_crawled`:

In [24]:
df[['date_crawled', 'time_crawled']] = df['date_crawled'].astype(str).str.split(pat=' ', n=1, expand=True)

df['date_crawled'] = pd.to_datetime(df['date_crawled'])
df['time_crawled'] = pd.to_datetime(df['time_crawled'], format='%H:%M:%S').dt.time

In [25]:
col_order = [
    'date_crawled', 'time_crawled', 'name', 'seller', 'offer_type', 'price_usd', 'abtest', 'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model', 'odometer_km', 'registration_month', 'fuel_type', 'brand', 'unrepaired_damage', 'date_created', 'postal_code', 'last_seen',
]
df = df[col_order]

df.head()

,date_crawled,time_crawled,name,seller,offer_type,price_usd,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,date_created,postal_code,last_seen
0,2016-03-26,17:47:46,Peugeot 807 160 NAVTECH ON BOARD,private,offer,5000.0,control,bus,2004,manual,158,others,150000.0,3,lpg,peugeot,no,2016-03-26,79588,2016-04-06 06:45:54
1,2016-04-04,13:38:56,BMW 740i 4 4 Liter HAMANN UMBAU Mega Optik,private,offer,8500.0,control,limousine,1997,automatic,286,7er,150000.0,6,petrol,bmw,no,2016-04-04,71034,2016-04-06 14:45:08
2,2016-03-26,18:57:24,Volkswagen Golf 1.6 United,private,offer,8990.0,test,limousine,2009,manual,102,golf,70000.0,7,petrol,volkswagen,no,2016-03-26,35394,2016-04-06 20:15:37
3,2016-03-12,16:58:10,Smart smart fortwo coupe softouch/F1/Klima/Pan...,private,offer,4350.0,control,small car,2007,automatic,71,fortwo,70000.0,6,petrol,smart,no,2016-03-12,33729,2016-03-15 03:16:28
4,2016-04-01,14:38:50,Ford Focus 1 6 Benzin TÜV neu ist sehr gepfleg...,private,offer,1350.0,test,station wagon,2003,manual,0,focus,150000.0,7,petrol,ford,no,2016-04-01,39218,2016-04-01 14:38:50


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49289 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date_crawled        49289 non-null  datetime64[ns]
 1   time_crawled        49289 non-null  object        
 2   name                49289 non-null  object        
 3   seller              49289 non-null  object        
 4   offer_type          49289 non-null  object        
 5   price_usd           49289 non-null  float64       
 6   abtest              49289 non-null  object        
 7   vehicle_type        49289 non-null  object        
 8   registration_year   49289 non-null  int64         
 9   gearbox             49289 non-null  object        
 10  power_ps            49289 non-null  int64         
 11  model               49289 non-null  object        
 12  odometer_km         49289 non-null  float64       
 13  registration_month  49289 non-null  int64     

**Dealing with Incorrect Registration Year Data**

Replace registration year with crawled year where the registration year is higher than the crawled year. This is because, we don't expect cars to be registered after they are posted:

In [42]:
reg_index = df.assign(
    year_crawled=lambda x: x.date_crawled.dt.year,
    reg_greater_crawl=lambda x: np.where(
        x.year_crawled < x.registration_year,
        'no', 'yes'
    ),
).query('reg_greater_crawl=="no" & registration_year > year_crawled').index

df.loc[reg_index, 'registration_year'] = df.loc[reg_index, 'date_crawled'].dt.year

In [43]:
df.assign(
    year_crawled=lambda x: x.date_crawled.dt.year,
    reg_greater_crawl=lambda x: np.where(
        x.year_crawled < x.registration_year,
        'no', 'yes'
    ),
).query('reg_greater_crawl=="no" & registration_year > year_crawled')

,date_crawled,time_crawled,name,seller,offer_type,price_usd,abtest,vehicle_type,registration_year,gearbox,...,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,date_created,postal_code,last_seen,year_crawled,reg_greater_crawl


Replace all registration years less than 1950 with 1950:

In [50]:
df.loc[df.registration_year < 1950]

,date_crawled,time_crawled,name,seller,offer_type,price_usd,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,date_created,postal_code,last_seen
2221,2016-03-15,14:57:07,Sehr seltener Oldtimer Opel 1210 zum Restaurieren,private,offer,3350.0,control,others,1934,manual,0,others,5000.0,0,petrol,opel,yes,2016-03-15,49828,2016-04-06 06:17:51
2573,2016-03-19,22:51:25,Hanomag rekord 15k Suche ersatz teile,private,offer,3000.0,test,others,1934,manual,0,others,90000.0,1,petrol,sonstige_autos,no,2016-03-19,90489,2016-03-19 22:51:25
10556,2016-04-01,06:02:10,UNFAL Auto,private,offer,450.0,control,small car,1800,manual,1800,others,5000.0,2,petrol,mitsubishi,no,2016-04-01,63322,2016-04-01 09:42:30
11047,2016-03-08,20:50:10,Andere Simca 5 Fourgonette Kombilimousine,private,offer,17500.0,control,station wagon,1948,manual,0,others,60000.0,6,petrol,sonstige_autos,no,2016-03-08,47546,2016-04-05 21:15:42
11246,2016-03-26,19:49:59,Ford Model A Roadster Deluxe 1931,private,offer,27500.0,control,convertible,1931,manual,39,others,10000.0,7,petrol,ford,no,2016-03-26,9322,2016-04-06 09:46:59
11585,2016-03-11,21:48:36,Volkswagen VW Typ 82,private,offer,41900.0,test,convertible,1943,manual,0,others,100000.0,7,petrol,volkswagen,yes,2016-03-11,84174,2016-03-21 13:18:05
13963,2016-03-20,17:51:49,Mercedes Benz L1500S Wehrmacht / Luftwaffe F...,private,offer,26900.0,test,others,1941,manual,60,others,60000.0,7,petrol,mercedes_benz,no,2016-03-20,38723,2016-04-07 01:17:51
21416,2016-03-12,08:36:21,Essex super six Ford A,private,offer,16500.0,control,convertible,1927,manual,40,others,5000.0,5,petrol,ford,no,2016-03-12,74821,2016-03-15 12:45:12
21421,2016-03-05,17:45:32,Ford Business Coupe Hotrod Projekt.1937,private,offer,7000.0,test,coupe,1937,manual,85,others,5000.0,8,petrol,ford,yes,2016-03-05,8359,2016-04-07 10:44:39
22101,2016-03-09,16:51:17,BMW Andere,private,offer,11500.0,test,convertible,1929,manual,15,others,5000.0,1,petrol,bmw,yes,2016-03-09,70569,2016-04-07 06:17:11


In [51]:
df.loc[df.registration_year < 1950, 'registration_year'] = 1950

In [52]:
df.loc[df.registration_year < 1950]

,date_crawled,time_crawled,name,seller,offer_type,price_usd,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,date_created,postal_code,last_seen


### Exploratory Data Analysis

In [54]:
df.describe(include='object')

,time_crawled,name,seller,offer_type,abtest,vehicle_type,gearbox,model,fuel_type,brand,unrepaired_damage,postal_code
count,49289,49289,49289,49289,49289,49289,49289,49289,49289,49289,49289,49289
unique,20570,38109,2,2,2,8,2,245,7,40,2,6999
top,20:36:19,Ford Fiesta,private,offer,test,small car,manual,others,petrol,volkswagen,no,10115
freq,14,78,49288,49288,25381,15206,39016,6014,33948,10509,44426,109


In [56]:
df.describe()

,price_usd,registration_year,power_ps,odometer_km,registration_month
count,4.928900e+04,49289.000000,49289.000000,49289.000000,49289.000000
mean,9.981811e+03,2003.329465,117.187608,126029.134290,5.777577
std,4.845606e+05,7.435167,199.353677,39561.876425,3.687663
min,0.000000e+00,1950.000000,0.000000,5000.000000,0.000000
25%,1.200000e+03,1999.000000,72.000000,125000.000000,3.000000
50%,2.999000e+03,2003.000000,107.000000,150000.000000,6.000000
75%,7.300000e+03,2008.000000,150.000000,150000.000000,9.000000
max,1.000000e+08,2016.000000,17700.000000,150000.000000,12.000000


**Exploring Price by Brand**

**Average price for Top 10 Brands**

**Average mileage for Top 10 Brands**

**Review `name` column**

See if there are particular keywords in the `name` column that can be extracted as new columns

**Most common brand/model combinations**

**Find Patterns in Odometer Columns**

Split the `odometer_km` into groups, and use aggregation to see if average prices follow any patterns based on the mileage.

**Determine Relationship between Price and Un-repaired Damage**

How much cheaper are cars with damage than their non-damaged counterparts?